In [79]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/jialing/Downloads/ResidentPopulationAged15YearsandOverbyReligionMaritalStatusandSexGeneralHouseholdSurvey2015 (2).csv")

In [80]:
df.columns.tolist()

['Thousands',
 'Total_Total',
 'Total_Males',
 'Total_Females',
 'Single_Total',
 'Single_Males',
 'Single_Females',
 'Married_Total',
 'Married_Males',
 'Married_Females',
 'Widowed_Total',
 'Widowed_Males',
 'Widowed_Females',
 'Divorced_Separated_Total',
 'Divorced_Separated_Males',
 'Divorced_Separated_Females']

In [81]:
df.rename(columns={'Thousands': 'Religion Type'}, inplace=True)

In [82]:
df = df.drop(df.columns[[1, 2, 3,4, 7,10,13]], axis=1)

In [83]:
df.columns.tolist()

['Religion Type',
 'Single_Males',
 'Single_Females',
 'Married_Males',
 'Married_Females',
 'Widowed_Males',
 'Widowed_Females',
 'Divorced_Separated_Males',
 'Divorced_Separated_Females']

In [84]:
df

,Religion Type,Single_Males,Single_Females,Married_Males,Married_Females,Widowed_Males,Widowed_Females,Divorced_Separated_Males,Divorced_Separated_Females
0,Total,533.2,501.3,984.7,960.9,28.9,143.2,42.7,80.9
1,No Religion,128.9,107.6,175.4,155.7,4.2,10.6,8.7,13.4
2,With Religion,404.3,393.8,809.4,805.2,24.8,132.6,34,67.5
3,Buddhism,160.6,149.4,336.6,333.2,10.3,54.3,14.9,28.0
4,Taoism1/,45.4,42.2,107.3,95.4,4.2,22.8,3.8,5.2
5,Islam,84.3,70.8,129.9,128.3,5.2,20.4,6.6,14.4
6,Hinduism,18.8,17.3,60.2,53.4,0.9,7.0,1.5,3.5
7,Sikhism,1.5,1.7,3.5,3.4,na,0.9,0.2,0.3
8,Christianity,92.4,111.2,169.3,189.4,4.1,26.8,6.9,15.9
9,Catholic,33.5,37.1,60.0,69.3,1.5,10.2,3.1,6.2


In [85]:
# Melt all marital/gender columns into two columns: 'variable' and 'Count'
df_tidy = df.melt(
    id_vars='Religion Type',
    var_name='Status_Gender',
    value_name='Count'
)
df_tidy

,Religion Type,Status_Gender,Count
0,Total,Single_Males,533.2
1,No Religion,Single_Males,128.9
2,With Religion,Single_Males,404.3
3,Buddhism,Single_Males,160.6
4,Taoism1/,Single_Males,45.4
...,...,...,...
91,Sikhism,Divorced_Separated_Females,0.3
92,Christianity,Divorced_Separated_Females,15.9
93,Catholic,Divorced_Separated_Females,6.2
94,Other Christians,Divorced_Separated_Females,9.7


In [86]:
# Fix inconsistent names
df_tidy['Status_Gender'] = df_tidy['Status_Gender'].replace({
    'Divorced_Separated_Males': 'Divorced_Males',
    'Divorced_Separated_Females': 'Divorced_Females'
})

In [87]:
# Separate the 'Status_Gender' into two columns (Marital Status, Gender)
df_tidy[['Marital Status', 'Gender']] = df_tidy['Status_Gender'].str.split('_', expand=True)


In [88]:
df_tidy.drop(columns=['Status_Gender'], inplace=True)

In [89]:
df_tidy.rename(columns={'Count': 'Value (Thousands)'}, inplace=True)

In [90]:
df_tidy['Gender'] = df_tidy['Gender'].replace({
    'Males' : "Male",
    "Females" : "Female"})


In [91]:
df_tidy['Religion Type'] = df_tidy['Religion Type'].replace({
    "Taoism1/" : "Taoism"})

In [92]:
df_tidy.drop(index=0, inplace=True)
df_tidy
df_tidy.to_csv("tidy_religion_marital_status.csv", index=False)

In [93]:
df_tidy['Religion Type'].value_counts()

Religion Type
No Religion         8
With Religion       8
Buddhism            8
Taoism              8
Islam               8
Hinduism            8
Sikhism             8
Christianity        8
Catholic            8
Other Christians    8
Other Religions     8
Total               7
Name: count, dtype: int64

In [94]:
df_tidy['Religion Type'] = df_tidy['Religion Type'].replace({
    "No Religion" : "Atheism"})

In [95]:
df_filtered =  df_tidy[~df_tidy['Religion Type'].isin(['With Religion', 'Total'])].copy()

In [96]:
df_filtered.to_csv("tidy_religion_marital_status.csv", index=False)

In [ ]:
df_new = df_filtered.reset_index(drop=True)
df_new["Value"] = df_new["Value (Thousands)"] * 1000
df_new.drop(columns=["Value (Thousands)"], inplace=True)

,Religion Type,Marital Status,Gender,Value
0,Atheism,Single,Male,128900.0
1,Buddhism,Single,Male,160600.0
2,Taoism,Single,Male,45400.0
3,Islam,Single,Male,84300.0
4,Hinduism,Single,Male,18800.0
...,...,...,...,...
75,Sikhism,Divorced,Female,300.0
76,Christianity,Divorced,Female,15900.0
77,Catholic,Divorced,Female,6200.0
78,Other Christians,Divorced,Female,9700.0


In [ ]:
df_new.groupby(['Religion Type', 'Marital Status'])['Value'].sum().reset_index() / df_

,Religion Type,Marital Status,Value
0,Atheism,Divorced,22100.0
1,Atheism,Married,331100.0
2,Atheism,Single,236500.0
3,Atheism,Widowed,14800.0
4,Buddhism,Divorced,42900.0
5,Buddhism,Married,669800.0
6,Buddhism,Single,310000.0
7,Buddhism,Widowed,64600.0
8,Catholic,Divorced,9300.0
9,Catholic,Married,129300.0


In [109]:
df_grouped = df_new.groupby(['Religion Type', 'Marital Status'])['Value'].sum().reset_index()
df_grouped['Percentage'] = df_grouped['Value'] / df_grouped['Value'].sum() * 100
df_grouped['Percentage'] = df_grouped['Percentage'].round(2)


In [110]:
df_new

,Religion Type,Marital Status,Gender,Value
0,Atheism,Single,Male,128900.0
1,Buddhism,Single,Male,160600.0
2,Taoism,Single,Male,45400.0
3,Islam,Single,Male,84300.0
4,Hinduism,Single,Male,18800.0
...,...,...,...,...
75,Sikhism,Divorced,Female,300.0
76,Christianity,Divorced,Female,15900.0
77,Catholic,Divorced,Female,6200.0
78,Other Christians,Divorced,Female,9700.0


In [107]:
df_grouped

,Religion Type,Marital Status,Value,Percentage
0,Atheism,Divorced,22100.0,0.57
1,Atheism,Married,331100.0,8.51
2,Atheism,Single,236500.0,6.08
3,Atheism,Widowed,14800.0,0.38
4,Buddhism,Divorced,42900.0,1.10
5,Buddhism,Married,669800.0,17.21
6,Buddhism,Single,310000.0,7.97
7,Buddhism,Widowed,64600.0,1.66
8,Catholic,Divorced,9300.0,0.24
9,Catholic,Married,129300.0,3.32


In [111]:
# save sheet 1 and sheet 2 into a single excel file
with pd.ExcelWriter('tidy_religion_marital_status.xlsx') as writer:
    df_new.to_excel(writer, sheet_name='Tidy Data', index=False)
    df_grouped.to_excel(writer, sheet_name='Summary', index=False)

In [119]:
from scipy.stats import chi2_contingency

# Pivot to create contingency table
ct = df_grouped.pivot(index='Religion Type', columns='Marital Status', values='Value')

# Run chi-square test
chi2, p, dof, expected = chi2_contingency(ct)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected counts:\n", expected)



Chi2: 48580.398494398
p-value: 0.0
Degrees of freedom: 27
Expected counts:
 [[2.27236954e+04 3.57940916e+05 1.92304902e+05 3.15304864e+04]
 [4.08725789e+04 6.43819947e+05 3.45894327e+05 5.67131478e+04]
 [8.30382846e+03 1.30800907e+05 7.02732059e+04 1.15220586e+04]
 [2.31559906e+04 3.64750379e+05 1.95963308e+05 3.21303220e+04]
 [6.11227935e+03 9.62798890e+04 5.17266785e+04 8.48115316e+03]
 [1.72880521e+04 2.72319317e+05 1.46304425e+05 2.39882063e+04]
 [1.48521622e+04 2.33949472e+05 1.25690103e+05 2.06082633e+04]
 [2.93208972e+02 4.61859246e+03 2.48135358e+03 4.06844986e+02]
 [4.32295280e+02 6.80946324e+03 3.65840592e+03 5.99835556e+02]
 [1.22659087e+04 1.93211118e+05 1.03803291e+05 1.70196819e+04]]


# Chi-Square Test Report: Religion Type vs Marital Status

## Objective:
To test whether Religion Type and Marital Status are independent or if there is an association between them.

### Method:
A chi-square test of independence was performed on a contingency table of religion type and marital status counts. The observed counts were compared to the expected counts under the assumption of independence.

### Results:

> Chi-square statistic (χ²): 48580.398

> Degrees of freedom (dof): 27

> P-value: 0.0

> Expected counts: See output table from expected array.

### Interpretation:

- The chi-square statistic is extremely large, which indicates that the observed counts differ greatly from the counts we would expect if religion and marital status were independent.

- The p-value is effectively 0, far below the common significance threshold of 0.05. This means we reject the null hypothesis of independence.

- Conclusion: There is strong statistical evidence that marital status is associated with religion type. In other words, the distribution of marital statuses (e.g., married, divorced, single) varies significantly across different religions.

- While the chi-square test confirms that an association exists, it does not indicate which religions or marital statuses contribute most to this association.